# descargas

> Funciones para descargar, procesar y actualizar las fuentes de datos que se usan en la librería. 

In [ ]:
#| default_exp descargas

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
import geopandas as gpd
import requests
import xml.etree.ElementTree as ET
import os
from zipfile import ZipFile
import shutil
from pathlib import Path

## descarga_denue_inegi

Esta función descarga las bases actualizadas del [DENUE](https://www.inegi.org.mx/app/descarga/?ti=6), las integra en una única base de datos para todo el país y guarda el resultado en la carpeta de datos. Esta función está aquí únicamente para documentar el proceso y, eventualmente, para actualizar los datos del DENUE cuando haya una nueva publicación.


In [ ]:
#| export
def descarga_denue_inegi():
    DOWNLOADS_PATH = "datos/descargas/denue/"
    Path(DOWNLOADS_PATH).mkdir(parents=True, exist_ok=True)
    Path(DOWNLOADS_PATH + '/extracted').mkdir(parents=True, exist_ok=True)
    tree = ET.parse("datos/DescargaMasivaOD.xml")
    root = tree.getroot()
    datos = []
    for archivo in root.iter('Archivo'):
        s = archivo.text
        tipo = s.rsplit("/")[-1].split(".")[-2].split("_")[-1]
        if tipo == 'shp' and (s.rsplit("/")[-2].split("_")[0] not in ['denue', 'esenciales']):
            url = archivo.text
            year = archivo.text.rsplit("/")[-2].split("_")[0]
            fname = s.rsplit("/")[-1]
            if os.path.exists(DOWNLOADS_PATH + fname):
                pass
            else:
                r = requests.get(url, allow_redirects=True)
                open(DOWNLOADS_PATH + fname, 'wb').write(r.content)
            try:
                gdf = gpd.read_file(DOWNLOADS_PATH + fname)
            except:
                try:
                    zf = ZipFile(DOWNLOADS_PATH + fname)
                    for f in zf.namelist():
                        zinfo = zf.getinfo(f)
                        if (zinfo.is_dir()):
                            if f.split("/")[-2] == 'conjunto_de_datos':
                                shps = [n for n in zf.namelist() if (n.startswith(f) and not n.endswith('/'))]
                                for s in shps:
                                    basename = os.path.basename(s)
                                    source = zf.open(s)
                                    target = open(os.path.join('datos/descargas/denue/extracted/', basename), 'wb')
                                    with source, target:
                                        shutil.copyfileobj(source, target)
                    gdf = gpd.read_file(os.path.join('datos/descargas/denue/extracted/', basename.split(".")[0] + '.shp'))
                    [f.unlink() for f in Path('datos/descargas/denue/extracted/').glob("*") if f.is_file()] 
                except:
                    print(f'El archivo {fname} está corrupto')                
            campos = ['nom_estab', 'raz_social', 'codigo_act', 'nombre_act', 
                    'per_ocu', 'tipoCenCom', 'cve_ent', 'cve_mun', 'cve_loc', 
                    'ageb', 'manzana', 'geometry']
            gdf = gdf.loc[:,campos]
            gdf['cvegeo'] = gdf['cve_ent'] + gdf['cve_mun'] + gdf['cve_loc'] + gdf['ageb'] + gdf['manzana']
            gdf['year'] = year
            datos.append(gdf)
            
    denue_total = pd.concat(datos)
    return denue_total

In [ ]:
# df = descarga_denue_inegi()
# df.head()

El archivo denue_46112-46311_25022015_shp.zip está corrupto


## descarga_redes

Esta función descarga un Geopackage con la red de transporte importada de [OpenStreetMap](https://www.openstreetmap.org/#map=5/23.944/-102.579).

**Nota:** En este momento la red sólo está disponible para la Zona Metropolitana de la Ciudad de México.

In [ ]:
#| export
def descarga_redes():
    DOWNLOADS_PATH = "datos/descargas/"
    Path(DOWNLOADS_PATH).mkdir(parents=True, exist_ok=True)
    fname = 'red_zmvm.gpkg'
    if os.path.exists(DOWNLOADS_PATH + fname):
        print("El archivo ya está descargado")
    else:
        url = 'https://www.dropbox.com/s/0fq8e8v2axyxxoc/red_zmvm.gpkg?dl=1'
        r = requests.get(url, allow_redirects=True)
        open(DOWNLOADS_PATH + fname, 'wb').write(r.content)

In [ ]:
descarga_redes()

El archivo ya está descargado


## descarga_poligonos_ciudades

Esta función descarga las slhuetas de las 100 ciudades más grandes de México (SUN, 2016)

In [ ]:
#| export
def descarga_poligonos_ciudades():
    DOWNLOADS_PATH = "datos/descargas/"
    Path(DOWNLOADS_PATH).mkdir(parents=True, exist_ok=True)
    fname = 'silhuetas.shp.zip'
    if os.path.exists(DOWNLOADS_PATH + fname):
        print("El archivo ya está descargado")
    else:
        url = 'https://www.dropbox.com/s/kofmn5qws911ktg/silhueta_100_ciudades.zip?dl=1'
        r = requests.get(url, allow_redirects=True)
        open(DOWNLOADS_PATH + fname, 'wb').write(r.content)

In [ ]:
descarga_poligonos_ciudades()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()